In [ ]:
import cobra
import d3flux as d3f

In [ ]:
from concerto.helpers.load_model_from_git import load_model_from_git

av = load_model_from_git('Azotobacter')
syn = load_model_from_git('Synechococcus')
rt = load_model_from_git('Rhodosporidium')

In [ ]:
av_all = [i.id for i in av.metabolites]
syn_all = [i.id for i in syn.metabolites]
rt_all = [i.id for i in rt.metabolites]

In [ ]:
print(av_all[0:5])

We are interested in metabolites that are imported/exported between the organisms.

In [ ]:
# comparing IDs
# compartment _e == extra_cellular
av_em = [i for i in av_all if i[-1] == 'e']
syn_em = [i for i in syn_all if i[-1] == 'e']
rt_em = [i for i in rt_all if i[-1] == 'e']

In [ ]:
print(av_em[0:5])

In [ ]:
### Find metabolites that 'could' be missing from models

In [ ]:
def check_metab_in_org(to_inspect, organism_background):
    background_core = [i.rsplit('_', 1)[0] for i in organism_background]
    found_in_org = set()
    for met in to_inspect:
        met_core = met.rsplit('_', 1)[0]
        if met_core in background_core:
            found_in_org.add(met_core)
    return found_in_org

In [ ]:
check_metab_in_org(set(syn_em).difference(av_em), av_all)

In [ ]:
check_metab_in_org(set(av_em).difference(rt_em), rt_all)

In [ ]:
check_metab_in_org(set(syn_em).difference(rt_em), rt_all)

# Total number of external metabolites between all three organisms

In [ ]:
r = av.reactions[0]
m = av.metabolites[0]

In [ ]:
all_ext_m = set(av_em).union(syn_em).union(rt_em)
print(len(all_ext_m))

In [ ]:
# BUILD MODEL BY LOOPING THROUGH REACTIONS
# Function to creatae model for purposes of visualization with d3flux
def make_metabolite_exchange_model(org_list):
    metabolite_exchange_model = cobra.Model("Metabolite_Exchange_Model")
    # go through each organism
    for this_org in org_list:
        with this_org as org:
            # create new metabolite species (needed for d3flux)
            org_m = cobra.Metabolite(org.id)
            org_m.compartment = org.id
            
            # go through each transport reaction with an external compartment
            org_transport_er = [r for r in org.reactions if ('e' in r.compartments and len(r.compartments)>1)]
            for r in org_transport_er:
                # go through each non-external metabolite
                for m in r.metabolites:
                    # filter out exchange reaction (only allow transport reaction)
                    # AND make sure we haven't already modified the internal metabolite of interest
                    # Note: exchange reactions only contain external metabolites
                    # modify metabolite compartment and id to include organism id
                    if (m.compartment != 'e') and (org.id not in m.compartment) :
                        m.compartment = m.compartment + "_" + org.id
                        m.id = m.id + "_" + org.id
                        # Add reaction between organism and metabolite
                        new_r = cobra.Reaction("Reaction_" + m.id)
                        new_r.add_metabolites({org_m:-1, m:1})
                        metabolite_exchange_model.add_reactions([new_r])
                # Change reaction id then add it to the new model
                r.id = r.id + "_" + org.id
                metabolite_exchange_model.add_reactions([r])
    return metabolite_exchange_model



In [ ]:
syn_1 = load_model_from_git('Synechococcus')
syn_1.id = "Se_1"

syn_2 = load_model_from_git('Synechococcus')
syn_2.id = "Se_2"

syn_exchange_model = make_metabolite_exchange_model([syn_1, syn_2])

In [ ]:
d3f.flux_map(syn_exchange_model)

In [ ]:
d3f.flux_map(syn_exchange_model)

In [ ]:
syn_exchange_model.reactions[2]




In [ ]:
# BUILD MODEL BY LOOPING THROUGH EXTERNAL METABOLITES ONLY

# Function to create model for purposes of visualization with d3flux
def make_metabolite_exchange_model_only_shared_external_metabolites(org_list):
    metabolite_exchange_model = cobra.Model("Metabolite_Exchange_Model")
    # go through each organism
    for this_org in org_list:
        with this_org as org:
            # create new metabolite species (needed for d3flux)
            org_m = cobra.Metabolite(org.id)
            org_m.compartment = org.id
            
            # go through each non-external metabolite
            org_m_reactions = []
            org_em = [m for m in org.metabolites if m.compartment == 'e']
            for m in org_em:
                # Add new reaction for this metabolite
                new_r = cobra.Reaction("Reaction_" + m.id + "_" + org.id)
                new_r.add_metabolites({org_m:-1, m:1})
                org_m_reactions.append(new_r)

            # Add new reactions to new model
            metabolite_exchange_model.add_reactions(org_m_reactions)
    
    lone_metabolites = [m for m in metabolite_exchange_model.metabolites if len(m.reactions)==1]
    metabolite_exchange_model.remove_metabolites(lone_metabolites, destructive=True)

    return metabolite_exchange_model


In [ ]:
av = load_model_from_git('Azotobacter')
syn = load_model_from_git('Synechococcus')
rt = load_model_from_git('Rhodosporidium')
syn.id = "Se_Se"
av.id = "Av_Av"
rt.id = "Rt_Rt"
av_syn_rt_exchange_model = make_metabolite_exchange_model_only_shared_external_metabolites([av, syn, rt])

d3f.flux_map(av_syn_rt_exchange_model)
